In [2]:
import os

In [5]:
pip install gitpython

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install langchain

  Using cached langchain-0.2.17-py3-none-any.whl.metadata (7.1 kB)
  Using cached PyYAML-6.0.2-cp38-cp38-win_amd64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.10.11-cp38-cp38-win_amd64.whl.metadata (8.0 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.5.0-cp38-cp38-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp38-cp38-win_amd64.whl.metadata (5.1 kB)
  Using cached

In [9]:
pip install langchain-community


  Using cached langchain_community-0.2.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.22.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.19-py3-none-any.whl (2.3 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.22.0-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)


In [10]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [11]:
%pwd

'c:\\Users\\This PC\\source-code-detector-using-GENAI\\research'

In [12]:
!mkdir test_repo

In [13]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/poonamDS1702/source-code-detector-using-GENAI", to_path=repo_path)

<git.repo.base.Repo 'c:\\Users\\This PC\\source-code-detector-using-GENAI\\research\\test_repo\\.git'>